In [7]:
import pandas as pd
import numpy as np
import nltk
import spacy

import sklearn
import jsonlines
import re
import os
from datetime import datetime
import json




In [8]:
def flatten_dict(pyobj, keystring=''): 
    if type(pyobj) == dict: 
        keystring = keystring + '_' if keystring else keystring 
        for k in pyobj: 
            yield from flatten_dict(pyobj[k], keystring + str(k)) 
    else: 
        yield keystring, pyobj 

In [9]:
def get_hashtags(hash_list):
    hashtags = ''
    for item in hash_list:
        hashtags = hashtags + ' ' + item['text']
        hashtags = hashtags.strip()
    return hashtags

In [10]:
key_names = ['lang', 'quoted_status_lang', 'created_at', 'id_str', 'full_text', 
             'entities_hashtags', 'entities_urls', 'entities_media',
             'quoted_status_full_text']

In [11]:
my_file = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-00.jsonl'
my_file2 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-01.jsonl'
my_file3 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-02.jsonl'
my_file4 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-03.jsonl'
my_file5 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-09.jsonl'

In [12]:
data = []



for file in [my_file, my_file2]:
    with jsonlines.open(my_file) as f:
        for line in f:
            # keep only if english and not a retweet
            if line['lang'] == 'en':
                data.append(dict(flatten_dict(line)))
    print('Loaded {} records'.format(len(data)))

Loaded 47409 records
Loaded 94818 records


In [24]:
key_names = data[0].keys()
tweets_df = pd.DataFrame(data, columns=key_names)

In [46]:
def remove_names_url(x):
    x = re.sub(r'http\S+', '', x) ## remove any urls in the tweets
    x = re.sub(r'@\S+','', x) #remove any usernames in the tweets
    return x

def is_retweet(x):
    if (x[0:2] == "RT"):
        return "retweet"
    else:
        return "original"


In [52]:
tweets_df['is_retweet'] =  tweets_df['full_text'].apply(is_retweet)

drop_index = tweets_df[tweets_df['is_retweet'] == "retweet"].index

no_retweets = tweets_df.drop(drop_index)

no_retweets['no_names'] = no_retweets['full_text'].apply(remove_names_url)



In [66]:
dir(no_retweets['no_names'].value_counts()[1:10])


['T',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__

In [65]:
no_retweets['user_screen_name'].value_counts()[1:10]

TheArticleTrunk    26
Gregory9           24
ShanghaiBeat       24
ktinfl             22
Frida_Goman        22
paul_furber        20
SecretNews         20
r__worldnews       20
FafaFalun          18
Name: user_screen_name, dtype: int64

In [90]:
 no_retweets['no_names'].value_counts()


Coronavirus Contains "HIV Insertions", Stoking Fears Over Artificially Created Bioweapon | Zero Hedge                                                                                                                          120
All of these stories — including one claiming that drinking chlorine dioxide will help get rid of coronavirus — were shared widely on social media. That doesn't mean they're true. Here are the real facts.                    74
Mountain Lion Throws Shade | Coronavirus Havoc | Patch PM                                                                                                                                                                       50
Novel Coronavirus: First Bay Area Case Confirmed                                                                                                                                                                                48
U.S. Declares Public Health Emergency Over Coronavirus                                      

In [118]:
delete_strings = no_retweets['no_names'].value_counts()[0:3].index.tolist()


In [43]:
tweets_df['full_text'][1][0:2]

'RT'

In [108]:
#no_retweets['full_text'][my_index]

In [114]:
delete_strings

['Coronavirus Contains "HIV Insertions", Stoking Fears Over Artificially Created Bioweapon | Zero Hedge ',
 "All of these stories — including one claiming that drinking chlorine dioxide will help get rid of coronavirus — were shared widely on social media. That doesn't mean they're true. Here are the real facts. ",
 'Mountain Lion Throws Shade | Coronavirus Havoc | Patch PM ',
 'Novel Coronavirus: First Bay Area Case Confirmed ',
 'U.S. Declares Public Health Emergency Over Coronavirus  ',
 'US First, Then China, Now Trump Pivots To The EU - Episode 2085a  via ']

In [119]:
copied_index = no_retweets['no_names'].isin(delete_strings)

In [125]:
no_retweets['full_text'][copied_index][60:80]


## most of these users don't look like they're bots. Some were suspended, but most seemed like they were somewhat "real"
## Most of the common names were headlines, so it may be smart to mark tweets that seem like they were made from headlines. 

36835    Mountain Lion Throws Shade | Coronavirus Havoc...
36836    Mountain Lion Throws Shade | Coronavirus Havoc...
36837    Mountain Lion Throws Shade | Coronavirus Havoc...
36838    Mountain Lion Throws Shade | Coronavirus Havoc...
36840    Mountain Lion Throws Shade | Coronavirus Havoc...
36841    Mountain Lion Throws Shade | Coronavirus Havoc...
36842    Mountain Lion Throws Shade | Coronavirus Havoc...
36843    Mountain Lion Throws Shade | Coronavirus Havoc...
36844    Mountain Lion Throws Shade | Coronavirus Havoc...
36845    Mountain Lion Throws Shade | Coronavirus Havoc...
36846    Mountain Lion Throws Shade | Coronavirus Havoc...
36847    Mountain Lion Throws Shade | Coronavirus Havoc...
36881    Coronavirus Contains "HIV Insertions", Stoking...
36975    Coronavirus Contains "HIV Insertions", Stoking...
37004    All of these stories — including one claiming ...
37005    All of these stories — including one claiming ...
37008    All of these stories — including one claiming .